<a href="https://colab.research.google.com/github/BernardoRMendes/Projeto/blob/main/Hedge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

Nesse projeto iremos verificar quais ativos servem de Hedge para o Ibovespa ao longo de toda a distribuição de retornos calculando a correlação condicional. Os dados foram retirados da ferramenta Profit e colocados em formato CSV.

In [ ]:
url="https://raw.githubusercontent.com/BernardoRMendes/Projeto/main/Hedge.csv"

In [ ]:
df=pd.read_csv(url,sep=';',index_col=0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1660 entries, 22/01/2021 to 30/04/2014
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   IBOVESPA  1660 non-null   object
 1   IVVB11    1660 non-null   object
 2   DÓLAR     1660 non-null   object
 3   IFIX      1660 non-null   object
 4   OZ1D      1660 non-null   object
dtypes: object(5)
memory usage: 77.8+ KB


In [ ]:
df

,IBOVESPA,IVVB11,DÓLAR,IFIX,OZ1D
Data,,,,,
22/01/2021,-0.008,0.0137,0.0214,-0.0027,0.0063
21/01/2021,-0.011,0.0112,0.0025,0.0001,0.0159
20/01/2021,-0.0082,0.005,0.0017,0.0014,-0.0031
19/01/2021,-0.005,0.0237,0.003,0.0021,0.0086
18/01/2021,0.0074,-0.0012,0.0014,0.0008,0.0139
...,...,...,...,...,...
07/05/2014,-0.0102,-0.0002,0.0078,0.0013,-0.0108
06/05/2014,0.0094,0.0014,-0.0015,0.0027,-0.0108
05/05/2014,-0.0027,-0.0045,0.0019,"0,0000",0.0022


In [ ]:
df.replace("0,0000","0.0000",inplace=True)

In [ ]:
df['IBOVESPA']=df['IBOVESPA'].astype(float)
df['IVVB11']=df['IVVB11'].astype(float)
df['DÓLAR']=df['DÓLAR'].astype(float)
df['IFIX']=df['IFIX'].astype(float)
df['OZ1D']=df['OZ1D'].astype(float)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1660 entries, 22/01/2021 to 30/04/2014
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   IBOVESPA  1660 non-null   float64
 1   IVVB11    1660 non-null   float64
 2   DÓLAR     1660 non-null   float64
 3   IFIX      1660 non-null   float64
 4   OZ1D      1660 non-null   float64
dtypes: float64(5)
memory usage: 77.8+ KB


Vamos colocar os retornos do Ibovespa em forma crescente e calcular a função de distribuição acumulada (CDF) empírica.

In [ ]:
df2=df.sort_values(by=['IBOVESPA'])

In [ ]:
df2.head(20)

,IBOVESPA,IVVB11,DÓLAR,IFIX,OZ1D
Data,,,,,
12/03/2020,-0.1478,-0.0873,0.0448,-0.0657,-0.0218
16/03/2020,-0.1392,-0.0773,0.0445,-0.0464,-0.0163
09/03/2020,-0.1217,-0.0604,0.0198,-0.0355,0.0373
18/03/2020,-0.1035,-0.0220,0.0121,-0.1323,-0.0078
18/05/2017,-0.0880,0.0858,0.0129,-0.0301,-0.0073
11/03/2020,-0.0764,-0.0337,0.0010,-0.0183,0.0244
26/02/2020,-0.0700,-0.0587,0.0099,-0.0149,0.0272
27/03/2020,-0.0551,-0.0124,0.0220,0.0105,0.0231
24/04/2020,-0.0545,0.0368,0.0375,-0.0247,-0.0307


In [ ]:
df2["CDF"]=1/1660
df2["CDF"]=df2["CDF"].cumsum()

In [ ]:
df2

,IBOVESPA,IVVB11,DÓLAR,IFIX,OZ1D,CDF
Data,,,,,,
12/03/2020,-0.1478,-0.0873,0.0448,-0.0657,-0.0218,0.000602
16/03/2020,-0.1392,-0.0773,0.0445,-0.0464,-0.0163,0.001205
09/03/2020,-0.1217,-0.0604,0.0198,-0.0355,0.0373,0.001807
18/03/2020,-0.1035,-0.0220,0.0121,-0.1323,-0.0078,0.002410
18/05/2017,-0.0880,0.0858,0.0129,-0.0301,-0.0073,0.003012
...,...,...,...,...,...,...
17/03/2016,0.0660,-0.0196,0.0047,0.0113,-0.0453,0.997590
10/03/2020,0.0714,0.0375,-0.0145,0.0142,-0.0379,0.998193
25/03/2020,0.0750,-0.0063,-0.0001,0.0582,-0.0308,0.998795


In [ ]:
a=pd.Series((df2.loc[(df2['CDF']>=0.00) & (df2['CDF']<0.01)].corr().values[0,:5]))
b=pd.Series((df2.loc[(df2['CDF']>=0.01) & (df2['CDF']<0.05)].corr().values[0,:5]))
c=pd.Series((df2.loc[(df2['CDF']>=0.05) & (df2['CDF']<0.1)].corr().values[0,:5]))
d=pd.Series((df2.loc[(df2['CDF']>=0.1) & (df2['CDF']<0.2)].corr().values[0,:5]))
e=pd.Series((df2.loc[(df2['CDF']>=0.2) & (df2['CDF']<0.3)].corr().values[0,:5]))
f=pd.Series((df2.loc[(df2['CDF']>=0.3) & (df2['CDF']<0.4)].corr().values[0,:5]))
g=pd.Series((df2.loc[(df2['CDF']>=0.4) & (df2['CDF']<0.5)].corr().values[0,:5]))
h=pd.Series((df2.loc[(df2['CDF']>=0.5) & (df2['CDF']<0.6)].corr().values[0,:5]))
i=pd.Series((df2.loc[(df2['CDF']>=0.6) & (df2['CDF']<0.7)].corr().values[0,:5]))
j=pd.Series((df2.loc[(df2['CDF']>=0.7) & (df2['CDF']<0.8)].corr().values[0,:5]))
k=pd.Series((df2.loc[(df2['CDF']>=0.8) & (df2['CDF']<0.9)].corr().values[0,:5]))
l=pd.Series((df2.loc[(df2['CDF']>=0.9) & (df2['CDF']<=1)].corr().values[0,:5]))

In [ ]:
lista=["[0,1)","[1,5)","[5,10)","[10,20)","[20,30)","[30,40)","[40,50)","[50,60)","[60,70)","[70,80)","[80,90)","[90,100]"]

In [ ]:
CondCorr=pd.concat([a,b,c,d,e,f,g,h,i,j,k,l],axis=1)

In [ ]:
CondCorr.columns=lista

In [ ]:
CondCorr.index=["IBOVESPA","IVVB11","DÓLAR","IFIX","OZ1D"]

As colunas representam a correlação condicional do Ibovespa em cada intervalo de retornos. Por exemplo, quando olhamos a coluna [0,1),isso representa os 1% piores retornos do Ibovespa no intervalo analisado, enquanto a coluna [90,100] é o intervalo que representa os retornos diários mais altos nesse período.

In [ ]:
CondCorr

,"[0,1)","[1,5)","[5,10)","[10,20)","[20,30)","[30,40)","[40,50)","[50,60)","[60,70)","[70,80)","[80,90)","[90,100]"
IBOVESPA,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
IVVB11,0.536217,-0.208783,0.091867,0.030027,0.069251,-0.090100,0.164692,0.046939,-0.005610,-0.028706,-0.092236,0.376695
DÓLAR,-0.650291,0.037538,-0.215888,0.058223,0.003008,-0.112224,0.029355,-0.001880,-0.214711,-0.145781,0.094463,-0.184469
IFIX,0.628925,0.252336,0.032307,0.064481,-0.008468,-0.014665,0.147744,-0.016717,0.016590,0.067139,0.069742,0.453175
OZ1D,0.196728,0.159675,0.072896,-0.033551,-0.015368,-0.197805,-0.003716,-0.036875,-0.010795,0.037407,0.031270,-0.074047


Essa é a matriz de correlação incondicional (avaliando a amostra inteira):

In [ ]:
df.corr()

,IBOVESPA,IVVB11,DÓLAR,IFIX,OZ1D
IBOVESPA,1.000000,0.143523,-0.138658,0.455501,-0.078450
IVVB11,0.143523,1.000000,-0.036858,0.097048,0.070795
DÓLAR,-0.138658,-0.036858,1.000000,-0.151042,0.043629
IFIX,0.455501,0.097048,-0.151042,1.000000,-0.040793
OZ1D,-0.078450,0.070795,0.043629,-0.040793,1.000000


Veja que o Ouro (OZ1D), apesar de apresentar correlação negativa em relação ao Ibovespa quando consideramos a amostra inteira, tem correlação positiva quando o Ibovespa tem seus piores dias, conforme podemos observar nos intervalos [0,1),[1,5),[5,10). No entanto, apresenta correlação negativa em quedas moderadas, como nos intervalos [10,20),[20,30),[30,40). Desse modo, podemos concluir que o Ouro é um bom Hedge em cenários de quedas moderadas, ao passo que não gera valor a um portfólio composto pelo Ibovespa em movimentos mais extremos. Para estes, o melhor Hedge, dentre os ativos considerados, é o Dólar.